In [1]:
# ! pip install numpy pandas neat-python graphviz pillow matplotlib

# Creating a dataset

In [78]:
import numpy as np
import pandas as pd

# Generate random data
np.random.seed(0)
num_samples = 100
marital_status = np.random.choice(['Single', 'Married'], num_samples)
previous_loan_returned = np.random.choice(['Yes', 'No'], num_samples)
annual_income = np.random.randint(20000, 100000, num_samples)
loan_approved = np.random.choice(["A", "B"], num_samples)

data = pd.DataFrame({
    'Marital Status': marital_status,
    'Previous Loan Returned': previous_loan_returned,
    'Annual Income': annual_income,
    'Loan Approved': loan_approved
})

# Update 'Loan Approved' based on the predefined condition
data['Loan Approved'] = np.where(
    (data['Annual Income'] > 40000) & 
    (data['Marital Status'] == 'Married') & 
    (data['Previous Loan Returned'] == 'Yes'), 
    "A", "B"  # 1 for Approved, 0 for Not Approved
)
  

df = data
df.head()

,Marital Status,Previous Loan Returned,Annual Income,Loan Approved
0,Single,No,53538,B
1,Married,Yes,28286,B
2,Married,Yes,38728,B
3,Single,No,33640,B
4,Married,Yes,88627,A


# NEAT Model

## Define the NEAT Configuration:

In [79]:
# ! pip install neat-python

[github config file](https://github.com/CodeReclaimers/neat-python/blob/master/examples/memory-variable/config)

In [80]:
neatconfigtemplate = """
[NEAT]
fitness_criterion     = max
fitness_threshold     = 1.0
pop_size              = 150
reset_on_extinction   = False

[DefaultGenome]
num_inputs            = 3  
num_outputs           = 1  
num_hidden            = 0  
initial_connection    = full
feed_forward          = True 
activation_default    = sigmoid
activation_mutate_rate= 0.0
activation_options    = sigmoid
compatibility_disjoint_coefficient    = 1.0
compatibility_weight_coefficient      = 0.6

# some more parameters
aggregation_mutate_rate = 0.0
bias_init_mean          = 0.0
bias_init_stdev         = 1.0
bias_replace_rate       = 0.1
bias_mutate_rate        = 0.7
bias_mutate_power       = 0.5
bias_max_value          = 30.0
bias_min_value          = -30.0
response_init_mean      = 1.0
response_init_stdev     = 0.1
response_replace_rate   = 0.1
response_mutate_rate    = 0.1
response_mutate_power   = 0.1
response_max_value      = 30.0
response_min_value      = -30.0

weight_max_value        = 30
weight_min_value        = -30
weight_init_mean        = 0.0
weight_init_stdev       = 1.0
weight_mutate_rate      = 0.8
weight_replace_rate     = 0.1
weight_mutate_power     = 0.5
enabled_default         = True
enabled_mutate_rate     = 0.01

# Node and connection mutation probabilities
node_add_prob         = 0.2
node_delete_prob      = 0.2
conn_add_prob         = 0.5
conn_delete_prob      = 0.5

[DefaultSpeciesSet]
compatibility_threshold = 3.0

[DefaultStagnation]
species_fitness_func = max
max_stagnation       = 20

[DefaultReproduction]
elitism              = 2
survival_threshold   = 0.2
"""


In [81]:
def create_file_with_text(text, filename):
    import os

    # Add .txt extension if no extension is provided
    if '.' not in filename:
        filename += '.txt'

    # Check if the directory exists, create it if it doesn't
    directory = os.path.dirname(filename)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)

    # Write the text to the file
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(text)
    print(f"File '{filename}' has been created with the provided text.")

# Example Usage
text = neatconfigtemplate
filename = 'neat_config'  # This will create 'example.txt'
create_file_with_text(text, filename)


File 'neat_config.txt' has been created with the provided text.


In [82]:
import neat

# Define the configuration
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation, 'neat_config.txt')


## Define the Evaluation Function:

In [83]:
# # Define the evaluation function for genomes
# def evaluate_genomes(genomes, config):
#     for genome_id, genome in genomes:
#         net = neat.nn.FeedForwardNetwork.create(genome, config)

#         error = 0.0
#         for _, row in data.iterrows():
#             # Encode categorical variables
#             marital_status_encoded = 1.0 if row['Marital Status'] == 'Married' else 0.0
#             previous_loan_encoded = 1.0 if row['Previous Loan Returned'] == 'Yes' else 0.0
            
#             # Input values
#             input_data = [marital_status_encoded, previous_loan_encoded, row['Annual Income']]
            
#             # Target value
#             target = row['Loan Approved']
            
#             # Activate the network and calculate error
#             output = net.activate(input_data)[0]
#             # print(str(output) + "is output " + "for " + str(row))
#             error += (output - target) ** 2

#         # Assign the fitness to the genome
#         # genome.fitness = 1.0 / (1.0 + error)
#         genome.fitness = error

In [84]:
# def evaluate_genomes(genomes, config):
#     for genome_id, genome in genomes:
#         net = neat.nn.FeedForwardNetwork.create(genome, config)

#         error = 0.0
#         for _, row in data.iterrows():
#             # Encode categorical variables
#             marital_status_encoded = 1.0 if row['Marital Status'] == 'Married' else 0.0
#             previous_loan_encoded = 1.0 if row['Previous Loan Returned'] == 'Yes' else 0.0
            
#             # Input values
#             input_data = [marital_status_encoded, previous_loan_encoded, row['Annual Income']]
            
#             # Target value (assuming 'A' is 1 and 'B' is 0)
#             target = 1.0 if row['Loan Approved'] == 'A' else 0.0
            
#             # Activate the network and calculate error
#             output = net.activate(input_data)[0]
#             error += (output - target) ** 2

#         # Assign the fitness to the genome
#         genome.fitness = 1.0 / (1.0 + error)

#         # Output the result of the current genome
#         output_label = 'A' if output >= 0.5 else 'B'
#         print(f"Genome {genome_id}: Output = {output_label}, Fitness = {genome.fitness}")


In [85]:
import numpy as np

def binary_cross_entropy(target, prediction):
    epsilon = 1e-5  # to avoid log(0)
    prediction = np.clip(prediction, epsilon, 1 - epsilon)
    return -(target * np.log(prediction) + (1 - target) * np.log(1 - prediction))

def evaluate_genomes(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)

        total_loss = 0.0
        for _, row in data.iterrows():
            # Encode categorical variables
            marital_status_encoded = 1.0 if row['Marital Status'] == 'Married' else 0.0
            previous_loan_encoded = 1.0 if row['Previous Loan Returned'] == 'Yes' else 0.0
            
            # Input values
            input_data = [marital_status_encoded, previous_loan_encoded, row['Annual Income']]
            
            # Target value (assuming 'A' is 1 and 'B' is 0)
            target = 1.0 if row['Loan Approved'] == 'A' else 0.0
            
            # Activate the network and calculate output
            output = net.activate(input_data)[0]

            # Calculate binary cross-entropy loss
            loss = binary_cross_entropy(target, output)
            total_loss += loss

        # Assign the average loss to the genome (lower is better)
        genome.fitness = -total_loss / len(data)

        # Output the result of the current genome
        output_label = 'A' if output >= 0.5 else 'B'
        # print(f"Genome {genome_id}: Output = {output_label}, Fitness = {genome.fitness}")


## Initialize and Evolve the Population:

In [86]:
# Create the population
p = neat.Population(config)

# Run the NEAT algorithm for a certain number of generations
for generation in range(10):
    p.run(evaluate_genomes, 1)
    print(f"Generation {generation+1}: Best Fitness = {p.best_genome.fitness}")

# Get the best genome after training
best_genome = p.best_genome
print(best_genome)

Generation 1: Best Fitness = -2.7631097116308534
Generation 2: Best Fitness = -0.5429137887556004
Generation 3: Best Fitness = -0.40528955475818434
Generation 4: Best Fitness = -0.3923406666483644
Generation 5: Best Fitness = -0.2680540868160595
Generation 6: Best Fitness = -0.20963747531896526
Generation 7: Best Fitness = -0.20963747531896526
Generation 8: Best Fitness = -0.20963747531896526
Generation 9: Best Fitness = -0.19012391924459937
Generation 10: Best Fitness = -0.17830256322445745
Key: 1434
Fitness: -0.17830256322445745
Nodes:
	0 DefaultNodeGene(key=0, bias=-1.8001836473232355, response=0.9957880439872941, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=0.8671333931196988, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=1.170951882808953, enabled=True)


In [87]:
# import graphviz
# import neat

# # Assume best_genome and config are already defined

# # Initialize a new Graphviz graph
# dot = graphviz.Digraph(format='png')
# # Create the neural network from the best genome
# best_net = neat.nn.FeedForwardNetwork.create(best_genome, config)

# # Add nodes and edges to the graph based on the best_genome structure
# for node_id in range(len(best_net.node_evals)):
#     dot.node(str(node_id))

# for conn_key, conn_gene in best_genome.connections.items():
#     if conn_gene.enabled:
#         # conn_key is a tuple in the form (in_node, out_node)
#         dot.edge(str(conn_key[0]), str(conn_key[1]), label=str(conn_gene.weight))

# # Save the Dot representation to a file
# dot.save('neat_network.gv')


In [88]:
import graphviz
import neat

# Assume best_genome and config are already defined

# Create the neural network from the best genome
best_net = neat.nn.FeedForwardNetwork.create(best_genome, config)

# Initialize a new Graphviz graph
dot = graphviz.Digraph(format='png')

# Define custom labels for nodes
input_labels = ['M', 'P', 'A']
output_label = 'O'

# Add input nodes
for i in range(len(input_labels)):
    dot.node(str(i), input_labels[i])

# Add hidden and output nodes
for node_id in range(len(input_labels), len(best_net.node_evals)):
    dot.node(str(node_id), output_label if node_id == len(best_net.node_evals) - 1 else '')

# Add edges
for conn_key, conn_gene in best_genome.connections.items():
    if conn_gene.enabled:
        # conn_key is a tuple in the form (in_node, out_node)
        dot.edge(str(conn_key[0]), str(conn_key[1]), label=str(conn_gene.weight))

# Save the Dot representation to a file
dot.save('neat_network.gv')


'neat_network.gv'

# using neat for predictions 

In [91]:
# Create the neural network from the best genome
best_net = neat.nn.FeedForwardNetwork.create(best_genome, config)

# Function to make predictions with the best network
def make_prediction(net, input_data):
    return net.activate(input_data)

# Example of making a prediction
# You need to encode your input data in the same way as during the training
marital_status_encoded = 1.0 # For example, 'Married'
previous_loan_encoded = 0.0  # For example, 'No'
annual_income = 10000         # Example income

# Make a prediction
prediction = make_prediction(best_net, [marital_status_encoded, previous_loan_encoded, annual_income])
print("Predicted Loan Approval Probability:", prediction[0])


Predicted Loan Approval Probability: 0.04027875121489312


In [ ]:
# Create the neural network from the best genome
best_net = neat.nn.FeedForwardNetwork.create(best_genome, config)

def draw_net(config, genome, view=False, filename=None, node_names=None, show_disabled=True, prune_unused=False,
             node_colors=None, fmt='svg'):
    """ Receives a genome and draws a neural network with arbitrary topology. """
    # Attributes for network nodes.
    if graphviz is None:
        warnings.warn("This display is not available due to a missing optional dependency (graphviz)")
        return

    if node_names is None:
        node_names = {}

    assert type(node_names) is dict

    if node_colors is None:
        node_colors = {}

    assert type(node_colors) is dict

    node_attrs = {
        'shape': 'circle',
        'fontsize': '9',
        'height': '0.2',
        'width': '0.2'}

    dot = graphviz.Digraph(format=fmt, node_attr=node_attrs)

    inputs = set()
    for k in config.genome_config.input_keys:
        inputs.add(k)
        name = node_names.get(k, str(k))
        input_attrs = {'style': 'filled', 'shape': 'box', 'fillcolor': node_colors.get(k, 'lightgray')}
        dot.node(name, _attributes=input_attrs)

    outputs = set()
    for k in config.genome_config.output_keys:
        outputs.add(k)
        name = node_names.get(k, str(k))
        node_attrs = {'style': 'filled', 'fillcolor': node_colors.get(k, 'lightblue')}

        dot.node(name, _attributes=node_attrs)

    if prune_unused:
        connections = set()
        for cg in genome.connections.values():
            if cg.enabled or show_disabled:
                connections.add((cg.in_node_id, cg.out_node_id))

        used_nodes = copy.copy(outputs)
        pending = copy.copy(outputs)
        while pending:
            new_pending = set()
            for a, b in connections:
                if b in pending and a not in used_nodes:
                    new_pending.add(a)
                    used_nodes.add(a)
            pending = new_pending
    else:
        used_nodes = set(genome.nodes.keys())

    for n in used_nodes:
        if n in inputs or n in outputs:
            continue

        attrs = {'style': 'filled',
                 'fillcolor': node_colors.get(n, 'white')}
        dot.node(str(n), _attributes=attrs)

    for cg in genome.connections.values():
        if cg.enabled or show_disabled:
            #if cg.input not in used_nodes or cg.output not in used_nodes:
            #    continue
            input, output = cg.key
            a = node_names.get(input, str(input))
            b = node_names.get(output, str(output))
            style = 'solid' if cg.enabled else 'dotted'
            color = 'green' if cg.weight > 0 else 'red'
            width = str(0.1 + abs(cg.weight / 5.0))
            dot.edge(a, b, _attributes={'style': style, 'color': color, 'penwidth': width})

    # Save the Dot representation to a file
    dot.save('neat_network.gv')